# 🏥 Core Health Check - Integration Tests

Notebook para verificar el estado de todos los componentes del módulo Core.

**Ubicación:** `core/test/integration/01_core_health_check.ipynb`

In [ ]:
# Setup inicial
import sys
import os
from pathlib import Path

# Agregar el directorio raíz al path
project_root = Path().absolute().parent.parent.parent
sys.path.insert(0, str(project_root))

print(f"📁 Proyecto root: {project_root}")
print(f"🐍 Python version: {sys.version}")

## 1. 🔍 Verificación de Imports

In [ ]:
def test_core_imports():
    """Verifica que todos los imports del core funcionen"""
    import_results = {}
    
    imports_to_test = [
        ("Postgres", "core.infraestructure.db.postgres", "Postgres"),
        ("S3Client", "core.infraestructure.aws.s3", "S3Client"),
        ("Domain Model", "core.models.domain", "Domain"),
        ("Event Model", "core.models.event", "Event"),
        ("Portfolio Model", "core.models.portfolio", "Portfolio"),
        ("DomainService", "core.services.domain_service", "DomainService"),
        ("EventService", "core.services.event_service", "EventService"),
        ("PortfolioService", "core.services.portfolio_service", "PortfolioService"),
        ("SQLAlchemyUnitOfWork", "core.infraestructure.uow.sqlalchemy_uow", "SQLAlchemyUnitOfWork")
    ]
    
    for name, module, class_name in imports_to_test:
        try:
            exec(f"from {module} import {class_name}")
            import_results[name] = "✅ OK"
            print(f"✅ {name} import OK")
        except Exception as e:
            import_results[name] = f"❌ ERROR: {str(e)}"
            print(f"❌ {name} import FAILED: {e}")
    
    return import_results

import_results = test_core_imports()

## 2. 🗄️ Test de Conexión a PostgreSQL

In [ ]:
def test_postgres_connection():
    """Test de conexión a PostgreSQL"""
    try:
        from core.infraestructure.db.postgres import Postgres
        from sqlalchemy import text
        
        print("🔌 Conectando a PostgreSQL...")
        db = Postgres()
        engine = db.connPostgres()
        
        if engine:
            print("✅ Conexión establecida")
            
            # Test básico de query
            with engine.connect() as conn:
                result = conn.execute(text("SELECT 1 as test, NOW() as current_time"))
                row = result.fetchone()
                print(f"✅ Query test OK - Result: {row[0]}, Time: {row[1]}")
                
                # Verificar tablas principales
                tables_query = text("""
                    SELECT table_name 
                    FROM information_schema.tables 
                    WHERE table_schema = 'public' 
                    AND table_name IN ('domains', 'events', 'portfolios')
                    ORDER BY table_name
                """)
                
                tables = conn.execute(tables_query).fetchall()
                print(f"📋 Tablas encontradas: {[t[0] for t in tables]}")
                
            return True, "PostgreSQL OK"
        else:
            return False, "No se pudo establecer conexión"
            
    except Exception as e:
        print(f"❌ Error: {e}")
        return False, str(e)

postgres_status, postgres_msg = test_postgres_connection()
print(f"\n📊 Estado PostgreSQL: {postgres_msg}")

## 3. ☁️ Test de Conexión a S3

In [ ]:
def test_s3_connection():
    """Test de conexión a S3"""
    try:
        from core.infraestructure.aws.s3 import S3Client
        
        print("☁️ Conectando a S3...")
        s3_client = S3Client()
        client = s3_client.conn_s3()
        
        if client:
            print("✅ Conexión S3 establecida")
            
            # Mostrar configuración
            config = s3_client.config
            print(f"📋 Bucket configurado: {config.get('bucket_name', 'No configurado')}")
            print(f"📋 Sufijo de archivo: {config.get('file_suffix', 'No configurado')}")
            
            # Intentar listar buckets (si hay permisos)
            try:
                buckets = s3_client.list_buckets()
                if buckets:
                    print(f"📋 Buckets disponibles: {buckets[:3]}...")  # Solo primeros 3
            except Exception as e:
                print(f"⚠️ No se pudieron listar buckets: {e}")
            
            return True, "S3 OK"
        else:
            return False, "No se pudo conectar a S3"
            
    except Exception as e:
        print(f"❌ Error S3: {e}")
        return False, str(e)

s3_status, s3_msg = test_s3_connection()
print(f"\n📊 Estado S3: {s3_msg}")

## 4. 🔧 Test del Unit of Work

In [ ]:
def test_unit_of_work():
    """Test del patrón Unit of Work"""
    try:
        from core.infraestructure.uow.sqlalchemy_uow import SQLAlchemyUnitOfWork
        
        print("🔧 Probando Unit of Work...")
        
        # Test de instanciación
        with SQLAlchemyUnitOfWork() as uow:
            print("✅ UoW instanciado correctamente")
            
            # Verificar repositorios disponibles
            repositories = ['domains', 'events', 'portfolios', 'portfolio_domains', 'vehicles', 'errors']
            available_repos = []
            
            for repo in repositories:
                if hasattr(uow, repo):
                    available_repos.append(repo)
                    print(f"✅ Repositorio '{repo}' disponible")
                else:
                    print(f"⚠️ Repositorio '{repo}' no encontrado")
            
            # Test de transacción simple
            uow.session.execute("SELECT 1")
            print("✅ Transacción de prueba OK")
            
            return True, f"UoW OK - Repositorios: {len(available_repos)}"
            
    except Exception as e:
        print(f"❌ Error UoW: {e}")
        return False, str(e)

uow_status, uow_msg = test_unit_of_work()
print(f"\n📊 Estado UoW: {uow_msg}")

## 5. ⚙️ Test de Servicios Core

In [ ]:
def test_core_services():
    """Test de instanciación de servicios core"""
    services_status = {}
    
    services_to_test = [
        ("DomainService", "core.services.domain_service", "DomainService"),
        ("EventService", "core.services.event_service", "EventService"),
        ("PortfolioService", "core.services.portfolio_service", "PortfolioService"),
        ("S3EventService", "core.services.s3_event_service", "S3EventService")
    ]
    
    for service_name, module, class_name in services_to_test:
        try:
            print(f"🔍 Probando {service_name}...")
            exec(f"from {module} import {class_name}")
            
            # Intentar instanciar (algunos pueden requerir parámetros)
            if service_name == "EventService":
                # EventService puede requerir UoW como parámetro
                exec(f"service = {class_name}()")
            else:
                exec(f"service = {class_name}()")
            
            services_status[service_name] = "✅ OK"
            print(f"✅ {service_name} instanciado OK")
            
        except Exception as e:
            services_status[service_name] = f"❌ {str(e)}"
            print(f"❌ {service_name} falló: {e}")
    
    return services_status

services_results = test_core_services()
print("\n📊 Resumen de Servicios:")
for service, status in services_results.items():
    print(f"  {service}: {status}")

## 6. 📊 Resumen Final del Health Check

In [ ]:
def generate_health_report():
    """Genera reporte final de salud del sistema"""
    print("🏥 REPORTE FINAL DE SALUD - MÓDULO CORE")
    print("=" * 50)
    
    # Contar éxitos
    total_components = 0
    successful_components = 0
    
    print("\n📋 COMPONENTES PRINCIPALES:")
    main_components = {
        "PostgreSQL": postgres_status,
        "S3": s3_status,
        "Unit of Work": uow_status
    }
    
    for component, status in main_components.items():
        total_components += 1
        if status:
            successful_components += 1
            print(f"  ✅ {component}: OK")
        else:
            print(f"  ❌ {component}: FAIL")
    
    print("\n📋 IMPORTS:")
    successful_imports = sum(1 for result in import_results.values() if "✅" in result)
    total_imports = len(import_results)
    print(f"  Exitosos: {successful_imports}/{total_imports}")
    
    print("\n📋 SERVICIOS:")
    successful_services = sum(1 for result in services_results.values() if "✅" in result)
    total_services = len(services_results)
    print(f"  Exitosos: {successful_services}/{total_services}")
    
    # Calcular porcentaje general
    total_checks = total_components + total_imports + total_services
    successful_checks = successful_components + successful_imports + successful_services
    success_rate = (successful_checks / total_checks) * 100 if total_checks > 0 else 0
    
    print(f"\n📊 SALUD GENERAL DEL SISTEMA: {success_rate:.1f}%")
    print(f"   ({successful_checks}/{total_checks} componentes funcionando)")
    
    if success_rate >= 80:
        print("\n🎉 SISTEMA EN BUEN ESTADO")
    elif success_rate >= 60:
        print("\n⚠️ SISTEMA CON PROBLEMAS MENORES")
    else:
        print("\n🚨 SISTEMA REQUIERE ATENCIÓN")
    
    return {
        'success_rate': success_rate,
        'total_checks': total_checks,
        'successful_checks': successful_checks,
        'details': {
            'main_components': main_components,
            'imports': import_results,
            'services': services_results
        }
    }

health_report = generate_health_report()

## 7. 💾 Guardar Reporte (Opcional)

In [ ]:
import json
from datetime import datetime

# Crear reporte JSON para logging
report_data = {
    'timestamp': datetime.now().isoformat(),
    'health_check_type': 'core_integration',
    'results': health_report
}

# Guardar en archivo (opcional)
# report_file = f"health_check_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
# with open(report_file, 'w') as f:
#     json.dump(report_data, f, indent=2)
# print(f"📄 Reporte guardado en: {report_file}")

print("✅ Health Check completado")
print(f"🕐 Ejecutado en: {report_data['timestamp']}")